# Exercise 2_3 Digital Image Processing

Amirkabir University of Technology

Dr. Rahmati

by Gholamreza Dar

Spring 2022

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import cv2
import os

sns.set_style("dark")


## Functions

In [ ]:
def load_images(path):
    """
    Loads images from the given path
    :param path: path to the images
    :return: list of images
    """
    images = []
    for filename in os.listdir(path):
        images.append(cv2.cvtColor(cv2.imread(os.path.join(path, filename)), cv2.COLOR_BGR2RGB).astype("float32"))
    return images

In [ ]:
def motion_blur_vertical(img, kernel_size = 30):
    # Create the vertical kernel.
    kernel_v = np.zeros((kernel_size, kernel_size))
    
    # Create a copy of the same for creating the horizontal kernel.
    kernel_h = np.copy(kernel_v)
    
    # Fill the middle row with ones.
    kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size)
    kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size)
    
    # Normalize.
    kernel_v /= kernel_size
    kernel_h /= kernel_size
    
    # Apply the vertical kernel.
    return cv2.filter2D(img, -1, kernel_v)

In [ ]:
def disp(img, title=None):
    plt.figure(figsize=(15,15))
    if title is not None:
        plt.title(title)
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    plt.show()

## Loading the images

In [ ]:
images = load_images("inputs/P3/frames")
test_1 = cv2.cvtColor(cv2.imread("inputs/P3/tests/pedestrians_test_01.png"), cv2.COLOR_BGR2RGB).astype('uint8')
test_2 = cv2.cvtColor(cv2.imread("inputs/P3/tests/pedestrians_test_02.png"), cv2.COLOR_BGR2RGB).astype('uint8')

## Extract the background

### Mean

In [ ]:
N_values = [2, 5, 10, 20]
backgrounds = []

# Calculate the background for each N
for N in N_values:
    backk = np.zeros((N, images[0].shape[0], images[0].shape[1], 3), dtype="float32")
    for i in range(N):
        backk[i] = images[i].astype("float32")
    backkk = np.mean(np.array(backk), axis=0)
    backgrounds.append(backkk.copy())

In [ ]:
# Dispaly the estimated backgrounds
fig, axs = plt.subplots(1, 4, figsize=(16, 3), constrained_layout=True)
fig.dpi = 100
fig.suptitle("Estimated Backgrounds (Mean)")
for i, ax in enumerate(axs.ravel()):
    ax.set_title(f"Average of {N_values[i]} images")
    ax.imshow(backgrounds[i].astype("uint8"))
    ax.set_axis_off()


### Median

In [ ]:
N_values = [2, 5, 10, 20]
backgrounds_median = []

# Calculate the background for each N
for N in N_values:
    backk = np.zeros((N, images[0].shape[0], images[0].shape[1], 3), dtype="float32")
    for i in range(N):
        backk[i] = images[i].astype("float32")
    backkk = np.median(np.array(backk), axis=0)
    backgrounds_median.append(backkk.copy())


In [ ]:
# Dispaly the estimated backgrounds
fig, axs = plt.subplots(1, 4, figsize=(16, 3), constrained_layout=True)
fig.dpi = 100
fig.suptitle("Estimated Backgrounds (Median)")
for i, ax in enumerate(axs.ravel()):
    ax.set_title(f"Mean of {N_values[i]} images")
    ax.imshow(backgrounds_median[i].astype("uint8"))
    ax.set_axis_off()


### Photoshop

In [ ]:
back = cv2.cvtColor(cv2.imread("back_ps.png"), cv2.COLOR_BGR2RGB)
disp(back, "Extracted background using Photoshop")

## Subtraction

In [ ]:
back = backgrounds_median[-1].astype('uint8')

sub = cv2.cvtColor(back, cv2.COLOR_RGB2GRAY).astype('float32') - cv2.cvtColor(test_1, cv2.COLOR_RGB2GRAY).astype('float32')

disp(back, "background")
disp(test_1, "test_1")
disp(sub, "subtracted")


In [ ]:
plt.figure(figsize=(15,15))
plt.axis('off')
plt.title("Subtracted")
plt.imshow(sub, cmap='gray', vmin=0,)

## Preprocessing

In [ ]:

_,res = cv2.threshold(sub,35,255,cv2.THRESH_TOZERO)
# res = cv2.cvtColor(res, cv2.COLOR_RGB2GRAY)
_,res = cv2.threshold(res,36,255,cv2.THRESH_BINARY)

# res = cv2.blur(res,(5, 5))

# res = cv2.erode(res, None, iterations=1)
# res = cv2.dilate(res, None, iterations=3)
# res = cv2.medianBlur(res.astype(np.uint8),27)

disp(res)

In [ ]:
np.unique(res)

In [ ]:
# D

test_1_masked = cv2.bitwise_and(test_1, test_1, mask=res.astype('uint8'))
disp(test_1_masked)

## Match Humans

### Method 1: Template matching

In [ ]:
template = cv2.imread('human_template3.png',0)
w, h = template.shape[::-1]

methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
            'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
methods = ['cv2.TM_CCOEFF']
for meth in methods:
    print(meth)
    match = cv2.matchTemplate(res.astype('uint8'), template, eval(meth))
    threshold = 6000000.0
    loc = np.where( match >= threshold)
    new_res = test_1.copy()
    people_center_map = np.zeros_like(res)

    random_index = np.random.randint(0, len(loc[0]), size=len(loc[0]))
    rects = []
    for i in range(len(loc[0])):
        pt = (loc[1][random_index[i]], loc[0][random_index[i]])
        # print(pt)
        is_near = False
        for rect in rects:
            # print(np.linalg.norm(np.array(rect)-np.array(pt)))
            if np.linalg.norm(np.array(rect)-np.array(pt)) < 145:
                is_near = True
        if not is_near:
            rects.append((pt[0], pt[1]))
            cv2.rectangle(new_res, pt, (pt[0] + w, pt[1] + h), (255, 0, 0), 2)
            cv2.rectangle(new_res, (pt[0]+w//2, pt[1]+h//2), (pt[0]+w//2+1, pt[1]+h//2+1), (0, 255, 0), 2)
            cv2.rectangle(people_center_map, (pt[0]+w//2, pt[1]+h//2), (pt[0]+w//2+1, pt[1]+h//2+1), (255, 255, 255), 2)

    disp(new_res)
    disp(people_center_map)
    disp(match)

### Method 2: Contours

In [ ]:
%%time
contours, hierarchy = cv2.findContours(res, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cont_res = test_1.copy()

i = 0
for cont in contours:
    if cv2.contourArea(cont) > 1200:
        i += 1
        cv2.drawContours(cont_res, [cont], -1, (0, 255, 0), 3)
        x,y,w,h = cv2.boundingRect(cont)
        text_width = 1 if i<10 else 2
        cv2.putText(cont_res, f"{i}", (x+w//2-text_width*16, y+h//2), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
        cv2.rectangle(cont_res, (x,y), (x+w, y+h), (0, 0, 255, 25), 4)
disp(cont_res)

## End to End 

In [ ]:
def detect_pedestrain(img, back, debug=False):
    min_contour_area = 2500

    # Calculate difference between image and background
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    back = cv2.cvtColor(back, cv2.COLOR_RGB2GRAY)
    sub = cv2.subtract(back, img_gray)
    
    sub = cv2.absdiff(back, img_gray)

    if debug:
        disp(sub, title="Subtraction")

    # Preprocessing [thresholding, blurring, eroding, dilating, blurring]
    _,res = cv2.threshold(sub,35,255,cv2.THRESH_TOZERO)
    _,res = cv2.threshold(res,36,255,cv2.THRESH_BINARY)
    res = cv2.blur(res,(5, 5))
    res = cv2.erode(res, None, iterations=1)
    res = cv2.dilate(res, None, iterations=3)
    res = cv2.medianBlur(res,27)

    if debug:
        disp(res, title="Preprocessed")
    
    # Find contours
    contours, hierarchy = cv2.findContours(res, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cont_res = img.copy()

    i = 0
    for cont in contours:
        # Only big contours
        if cv2.contourArea(cont) > min_contour_area:
            if debug:
                print(i, cv2.contourArea(cont))
            i += 1
            x,y,w,h = cv2.boundingRect(cont)
            text_width = 1 if i<10 else 2

            # visualize countours
            cv2.drawContours(cont_res, [cont], -1, (0, 255, 0), 3)
            cv2.rectangle(cont_res, (x,y), (x+w, y+h), (0, 0, 255, 25), 4)
            cv2.putText(cont_res, f"{i}", (x+w//2-text_width*16, y+h//2), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
            
    if debug:
        disp(cont_res, title="Contours")
        print(f"{i} pedestrians detected")

    cv2.putText(cont_res, f"{i} pedestrians detected", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 3)
    
    return cont_res

In [ ]:
back = cv2.imread("back_ps.png")

In [ ]:
result = detect_pedestrain(test_1, back.astype('uint8'), debug=True)
# disp(result, title="Pedestrians in Test_1")

In [ ]:
result = detect_pedestrain(test_2, back.astype("uint8"), debug=True)
# disp(result, title="Pedestrians in Test_2")

In [ ]:
result = detect_pedestrain(images[8].astype('uint8'), back, debug=True)
# disp(result, title="Pedestrians in Test_2")

In [ ]:
for i, image in enumerate(images):
    result = detect_pedestrain(image.astype('uint8'), back, debug=False)
    cv2.imwrite(f"P3_result/frame_{i}_result3.jpg", cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
    # disp(result, title="Pedestrians in Frame")